In [9]:
import tf_keras as keras
import tf_keras.backend as K
import numpy as np

import utils
import loggingreporter 

# Configurations of DNN
cfg = {}
cfg['SGD_BATCHSIZE']    = 128
cfg['SGD_LEARNINGRATE'] = 0.004
cfg['NUM_EPOCHS']       = 2001
cfg['FULL_MI']          = True

# choose activation functions
cfg['ACTIVATION'] = 'tanh'

# DNN architectures
# cfg['LAYER_DIMS'] = [1024, 20, 20, 20] 
cfg['LAYER_DIMS'] = [512, 256, 128, 64, 32, 16, 10, 10]  
# cfg['LAYER_DIMS'] = [10, 10, 10, 10] 

ARCH_NAME =  '-'.join(map(str,cfg['LAYER_DIMS']))

training_per = 1
trn, tst = utils.get_mnist(training_percent=training_per)
# trn, tst = utils.get_FashionMNIST(training_percent=training_per)

# Where to save activation and weights data
cfg['SAVE_DIR'] = 'rawdata/' + cfg['ACTIVATION'] + '_' + ARCH_NAME 

In [10]:
input_layer  = keras.layers.Input((trn.X.shape[1],))
clayer = input_layer
for n in cfg['LAYER_DIMS']:
    clayer = keras.layers.Dense(n, activation=cfg['ACTIVATION'])(clayer)
output_layer = keras.layers.Dense(trn.nb_classes, activation='softmax')(clayer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)
optimizer = keras.optimizers.legacy.SGD(learning_rate=cfg['SGD_LEARNINGRATE'])

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
def do_report(epoch):
    # Only log activity for some epochs.  Mainly this is to make things run faster.
    if epoch < 20:       # Log for all first 20 epochs
        return True
    elif epoch < 100:    # Then for every 5th epoch
        return (epoch % 5 == 0)
    elif epoch < 200:    # Then every 10th
        return (epoch % 10 == 0)
    else:                # Then every 100th
        return (epoch % 20 == 0)
    
reporter = loggingreporter.LoggingReporter(cfg=cfg, 
                                          trn=trn, 
                                          tst=tst, 
                                          do_save_func=do_report)

# change the printing way by setting the parameter "verbose"
r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 2, 
              batch_size = cfg['SGD_BATCHSIZE'],
              epochs     = cfg['NUM_EPOCHS'],
              validation_data=(tst.X, tst.Y),
              callbacks  = [reporter,])
print("done")

In [13]:
# save the losses and accuracy 
import pickle
losses = r.history
with open(f"{cfg['SAVE_DIR']}/losses.pickle", "wb") as f:
    pickle.dump(losses, f)